In [2]:
from bitcoincli import Bitcoin
import datetime

In [90]:
# creds

host = "172.33.0.2"
port = "8332"
username = "dappnode"
password = "dappnode"

bitcoin = Bitcoin(username, password, host, port)

info = bitcoin.getblockchaininfo()
print(info)

{'chain': 'main', 'blocks': 845355, 'headers': 845355, 'bestblockhash': '00000000000000000001ebb5f37072c22b4e3f27b70835e3271a8057440e0630', 'difficulty': 84381461788831.34, 'time': 1716802873, 'mediantime': 1716795515, 'verificationprogress': 0.9999987402328071, 'initialblockdownload': False, 'chainwork': '00000000000000000000000000000000000000007b97d581a1928b74fd57c708', 'size_on_disk': 652798159950, 'pruned': False, 'warnings': ''}


In [91]:
def get_latest_block():
    return bitcoin.getblockcount()

In [92]:
bheight = 830000
print("bheight", bheight)
bhash = bitcoin.getblockhash(bheight)

print("bhash", bhash)

bheight 830000
bhash 000000000000000000011d55599ed27d7efca05f5849b755319c89eb2cffbc1f


In [93]:
def get_tx(api, txid, blockhash):
    return api.getrawtransaction(txid, True, blockhash)

In [94]:
block_info = bitcoin.getblock(bhash)

In [95]:
txid = block_info['tx'][1]

In [96]:
txinfo = get_tx(bitcoin, txid, bhash)

In [97]:
import time

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Время выполнения функции '{func.__name__}': {execution_time:.6f} секунд")
        print(f"Аргументы: args={args}, kwargs={kwargs}")
        return result
    return wrapper

In [98]:
def ts_to_datetime(ts):
    return datetime.datetime.utcfromtimestamp(ts)


def block_info_to_datetime(block_info):
    return ts_to_datetime(block_info['time'])

@timing_decorator
def binary_search(getfn, target, left, right):
    while right - left > 1:
        middle = (left + right) // 2
        if getfn(middle) >= target:
            right = middle
        else:
            left = middle
    return left

In [99]:
ts_to_datetime(block_info['time'])

datetime.datetime(2024, 2, 11, 18, 17, 37)

In [100]:
dt_start = datetime.datetime(2024, 3, 13, 22, 59, 0)
dt_end = datetime.datetime(2024, 3, 18, 18, 34, 0)

In [104]:
def get_block_by_height(api, height):
    blockhash = api.getblockhash(height)
    return api.getblock(blockhash)

In [105]:
def block_date_by_height(height):
    block_info = get_block_by_height(bitcoin, height)
    return block_info_to_datetime(block_info)

In [106]:
search_start = 830000
search_end = 840000
print(block_date_by_height(search_start))
print(block_date_by_height(search_end))

2024-02-11 18:17:37
2024-04-20 00:09:27


In [107]:
start_block = binary_search(block_date_by_height, dt_start, search_start, search_end)
start_block

Время выполнения функции 'binary_search': 2.179212 секунд
Аргументы: args=(<function block_date_by_height at 0x787339cd3d00>, datetime.datetime(2024, 3, 13, 22, 59), 830000, 840000), kwargs={}


834577

In [108]:
end_block = binary_search(block_date_by_height, dt_end, search_start, search_end)
end_block = end_block + 1
end_block

Время выполнения функции 'binary_search': 2.480851 секунд
Аргументы: args=(<function block_date_by_height at 0x787339cd3d00>, datetime.datetime(2024, 3, 18, 18, 34), 830000, 840000), kwargs={}


835257

In [82]:
end_block - start_block

680

In [83]:
print(block_date_by_height(start_block))
print(block_date_by_height(end_block))

2024-03-13 22:40:42
2024-03-18 18:36:42


In [109]:
block_info.keys()

dict_keys(['hash', 'confirmations', 'height', 'version', 'versionHex', 'merkleroot', 'time', 'mediantime', 'nonce', 'bits', 'difficulty', 'chainwork', 'nTx', 'previousblockhash', 'nextblockhash', 'strippedsize', 'size', 'weight', 'tx'])

In [110]:
@timing_decorator
def get_block_txs(height):
    txs = []
    block_info = get_block_by_height(bitcoin, height)
    block_hash = block_info['hash']
    block_txs = block_info['tx']
    for txid in block_txs:
        txs.append(get_tx(bitcoin, txid, block_hash))
    return txs

In [114]:
start_block_txs = get_block_txs(start_block)

Время выполнения функции 'get_block_txs': 69.699919 секунд
Аргументы: args=(834577,), kwargs={}


In [ ]:
txs = []
for bn in range(start_block, end_block):
    txs.extend(get_block_txs(bn))